## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
from allennlp.data.token_indexers import TokenIndexer, PretrainedTransformerIndexer
from allennlp.data.tokenizers import Token, Tokenizer, PretrainedTransformerTokenizer

import nltk
#nltk.download('punkt')
import numpy as np
from os import listdir
from os.path import join as pathjoin
import torch
import torch.nn as nn
from torch.nn import functional as F
import tqdm

from minGPT.mingpt.model import GPT, GPTConfig
from minGPT.mingpt.trainer import Trainer, TrainerConfig
# make deterministic
from minGPT.mingpt.utils import sample, set_seed
set_seed(42)

In [2]:
DATA_DIR = '/home/mlepekhin/data/big'
#MODELS_DIR = '/home/mlepekhin/models/big'
transformer_model = 'DeepPavlov/rubert-base-cased'

In [3]:
import math
from torch.utils.data import Dataset


def detokenize(tokens):
    return ' '.join([str(x) for x in tokens[1:-1]]).replace(' ##', '')

class BPEDataset(Dataset):
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [4]:
block_size = 128
tokenizer = PretrainedTransformerTokenizer(transformer_model)
#indexer = PretrainedTransformerIndexer(transformer_model)

In [8]:
def train_gpt_generator(train_text_file, state_dict_file, n_layer=8, n_head=8, n_embd=512,
                        max_epochs=1, batch_size=256):
    text_sentences = nltk.tokenize.sent_tokenize(open(train_text_file, 'r').read())
    tokens = np.concatenate([tokenizer.tokenize(sent)[1:-1] for sent in text_sentences])
    tokens = [str(token) for token in tokens]
    train_dataset = BPEDataset(tokens, block_size) 
    
    mconf = GPTConfig(
        train_dataset.vocab_size, train_dataset.block_size,
        n_layer=n_layer, n_head=n_head, n_embd=n_embd
    )
    model = GPT(mconf)
    tconf = TrainerConfig(
        max_epochs=max_epochs, batch_size=batch_size, learning_rate=6e-4,
        lr_decay=True, warmup_tokens=batch_size*20, final_tokens=2*len(train_dataset)*block_size,
        num_workers=3
    )
    trainer = Trainer(model, train_dataset, None, tconf)
    trainer.train()
    torch.save(model.state_dict(), state_dict_file)

In [9]:
GENRE_DATA_DIR = '/home/mlepekhin/data/big/genre'
GPT_MODELS_DIR = '/home/mlepekhin/models/mini_gpt_big_bpe/'
LANG = 'ru'

In [10]:
#train_gpt_generator(
#        pathjoin(GENRE_DATA_DIR, LANG, 'A1.txt'),
#        pathjoin(GPT_MODELS_DIR, LANG, 'A1')
#)

In [11]:
for train_text_file in tqdm.tqdm(listdir(pathjoin(GENRE_DATA_DIR, LANG))):
    label = train_text_file[:-4]
    train_gpt_generator(
        pathjoin(GENRE_DATA_DIR, LANG, train_text_file),
        pathjoin(GPT_MODELS_DIR, LANG, label)
    )

  0%|          | 0/10 [00:00<?, ?it/s]

data has 732852 characters, 38863 unique.



  0%|          | 0/2863 [00:00<?, ?it/s]/home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

epoch 1 iter 0: train loss 10.66786. lr 6.000000e-04:   0%|          | 0/2863 [00:07<?, ?it/s]
epoch 1 iter 0: train loss 10.66786. lr 6.000000e-04:   0%|          | 1/2863 [00:07<6:16:40,  7.90s/it]
epoch 1 iter 1: train loss 9.92521. lr 5.999998e-04:   0%|          | 1/2863 [00:08<6:16:40,  7.90s/it] 
epoch 1 iter 1: train loss 9.92521. lr 5.999998e-04:   0%|          | 2/2863 [00:08<4:33:06,  5.73s/it]
epoch 1 iter 2: train loss 9.49199. lr 5.999996e-04:   0%|          | 2/2863 [00:09<4:33:06,  5.73s/it]
epoch 1 iter 2: train loss 9.49199. lr 5.999996e-04:   0%|          | 3/2863 [00:09<3:20:35,  4.21s/it]
epoch 1 iter 3: train loss 9.

epoch 1 iter 36: train loss 7.53150. lr 5.999387e-04:   1%|▏         | 37/2863 [00:32<34:18,  1.37it/s]
epoch 1 iter 37: train loss 7.51890. lr 5.999353e-04:   1%|▏         | 37/2863 [00:32<34:18,  1.37it/s]
epoch 1 iter 37: train loss 7.51890. lr 5.999353e-04:   1%|▏         | 38/2863 [00:32<34:38,  1.36it/s]
epoch 1 iter 38: train loss 7.47893. lr 5.999318e-04:   1%|▏         | 38/2863 [00:33<34:38,  1.36it/s]
epoch 1 iter 38: train loss 7.47893. lr 5.999318e-04:   1%|▏         | 39/2863 [00:33<34:44,  1.35it/s]
epoch 1 iter 39: train loss 7.53272. lr 5.999283e-04:   1%|▏         | 39/2863 [00:34<34:44,  1.35it/s]
epoch 1 iter 39: train loss 7.53272. lr 5.999283e-04:   1%|▏         | 40/2863 [00:34<34:53,  1.35it/s]
epoch 1 iter 40: train loss 7.46947. lr 5.999246e-04:   1%|▏         | 40/2863 [00:35<34:53,  1.35it/s]
epoch 1 iter 40: train loss 7.46947. lr 5.999246e-04:   1%|▏         | 41/2863 [00:35<35:04,  1.34it/s]
epoch 1 iter 41: train loss 7.43743. lr 5.999209e-04:   1%|▏    

epoch 1 iter 74: train loss 7.12664. lr 5.997470e-04:   3%|▎         | 75/2863 [01:00<35:15,  1.32it/s]
epoch 1 iter 75: train loss 7.16155. lr 5.997401e-04:   3%|▎         | 75/2863 [01:01<35:15,  1.32it/s]
epoch 1 iter 75: train loss 7.16155. lr 5.997401e-04:   3%|▎         | 76/2863 [01:01<35:05,  1.32it/s]
epoch 1 iter 76: train loss 7.12394. lr 5.997332e-04:   3%|▎         | 76/2863 [01:02<35:05,  1.32it/s]
epoch 1 iter 76: train loss 7.12394. lr 5.997332e-04:   3%|▎         | 77/2863 [01:02<35:09,  1.32it/s]
epoch 1 iter 77: train loss 7.14290. lr 5.997263e-04:   3%|▎         | 77/2863 [01:03<35:09,  1.32it/s]
epoch 1 iter 77: train loss 7.14290. lr 5.997263e-04:   3%|▎         | 78/2863 [01:03<35:08,  1.32it/s]
epoch 1 iter 78: train loss 7.13642. lr 5.997192e-04:   3%|▎         | 78/2863 [01:03<35:08,  1.32it/s]
epoch 1 iter 78: train loss 7.13642. lr 5.997192e-04:   3%|▎         | 79/2863 [01:03<35:03,  1.32it/s]
epoch 1 iter 79: train loss 7.07333. lr 5.997120e-04:   3%|▎    

...

epoch 1 iter 2800: train loss 0.27686. lr 3.100878e-04:  98%|█████████▊| 2801/2863 [33:50<00:48,  1.27it/s]
epoch 1 iter 2801: train loss 0.28917. lr 3.099233e-04:  98%|█████████▊| 2801/2863 [33:50<00:48,  1.27it/s]
epoch 1 iter 2801: train loss 0.28917. lr 3.099233e-04:  98%|█████████▊| 2802/2863 [33:50<00:47,  1.28it/s]
epoch 1 iter 2802: train loss 0.28973. lr 3.097587e-04:  98%|█████████▊| 2802/2863 [33:51<00:47,  1.28it/s]
epoch 1 iter 2802: train loss 0.28973. lr 3.097587e-04:  98%|█████████▊| 2803/2863 [33:51<00:47,  1.28it/s]
epoch 1 iter 2803: train loss 0.29727. lr 3.095942e-04:  98%|█████████▊| 2803/2863 [33:52<00:47,  1.28it/s]
epoch 1 iter 2803: train loss 0.29727. lr 3.095942e-04:  98%|█████████▊| 2804/2863 [33:52<00:46,  1.27it/s]
epoch 1 iter 2804: train loss 0.27966. lr 3.094296e-04:  98%|█████████▊| 2804/2863 [33:53<00:46,  1.27it/s]
epoch 1 iter 2804: train loss 0.27966. lr 3.094296e-04:  98%|█████████▊| 2805/2863 [33:53<00:45,  1.27it/s]
epoch 1 iter 2805: train los

epoch 1 iter 2837: train loss 0.28168. lr 3.039977e-04:  99%|█████████▉| 2837/2863 [34:17<00:18,  1.42it/s]
epoch 1 iter 2837: train loss 0.28168. lr 3.039977e-04:  99%|█████████▉| 2838/2863 [34:17<00:17,  1.43it/s]
epoch 1 iter 2838: train loss 0.28607. lr 3.038331e-04:  99%|█████████▉| 2838/2863 [34:18<00:17,  1.43it/s]
epoch 1 iter 2838: train loss 0.28607. lr 3.038331e-04:  99%|█████████▉| 2839/2863 [34:18<00:16,  1.44it/s]
epoch 1 iter 2839: train loss 0.26548. lr 3.036684e-04:  99%|█████████▉| 2839/2863 [34:19<00:16,  1.44it/s]
epoch 1 iter 2839: train loss 0.26548. lr 3.036684e-04:  99%|█████████▉| 2840/2863 [34:19<00:15,  1.44it/s]
epoch 1 iter 2840: train loss 0.26813. lr 3.035038e-04:  99%|█████████▉| 2840/2863 [34:19<00:15,  1.44it/s]
epoch 1 iter 2840: train loss 0.26813. lr 3.035038e-04:  99%|█████████▉| 2841/2863 [34:19<00:15,  1.44it/s]
epoch 1 iter 2841: train loss 0.28392. lr 3.033392e-04:  99%|█████████▉| 2841/2863 [34:20<00:15,  1.44it/s]
epoch 1 iter 2841: train los

data has 1895152 characters, 44695 unique.



 10%|█         | 1/10 [36:24<5:27:43, 2184.87s/it]


RuntimeError: CUDA out of memory. Tried to allocate 1.37 GiB (GPU 3; 23.65 GiB total capacity; 7.18 GiB already allocated; 180.12 MiB free; 8.32 GiB reserved in total by PyTorch) (malloc at /pytorch/c10/cuda/CUDACachingAllocator.cpp:289)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x46 (0x7fbfe9ccd536 in /home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x1cf1e (0x7fbfe9f16f1e in /home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/torch/lib/libc10_cuda.so)
frame #2: <unknown function> + 0x1df9e (0x7fbfe9f17f9e in /home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/torch/lib/libc10_cuda.so)
frame #3: at::native::empty_cuda(c10::ArrayRef<long>, c10::TensorOptions const&, c10::optional<c10::MemoryFormat>) + 0x135 (0x7fbfecaa59e5 in /home/mlepekhin/anaconda3/envs/mlepekhin_research/lib/python3.8/site-packages/torch/lib/libtorch_cuda.

1
